<a href="https://colab.research.google.com/github/marienams/TweetSentimentAnalysis/blob/main/GoogleColabNotebook/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# API to fetch dataset from kaggle

In [5]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 82% 66.0M/80.9M [00:00<00:00, 147MB/s]
100% 80.9M/80.9M [00:00<00:00, 136MB/s]


In [6]:
# extract dataset from zip file

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Data extracted from zip')

Data extracted from zip


# Importing the dependencies

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
#natural language toolkit lib
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Getting the list of stopwords

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Processing


## Declaring our data frame here

In [11]:
# loading data from csv file

column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',
                           names= column_names, encoding='ISO-8859-1')

# data display
# stemming

In [12]:
# checking rows and columns and counts and then doing a even distribution check
twitter_data.shape

(1600000, 6)

In [13]:
# 16mil rows and 6 col
# print first 5 rows

twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
# using columns_names allowed us to set the column names,
# otherwise the rows are 15.99999mil that is one less than 16mil


In [15]:
# counting number of missing values

In [16]:
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [17]:
# now even distribution

In [18]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [19]:
#data is evenoy distributed, between positive and negative

In [20]:
# 0= negative

In [22]:
# 4 = positive

## Stemming

Stemming is the process of reducing a word to its root words.

In [23]:
port_stem = PorterStemmer()

In [31]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [32]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [33]:
twitter_data.head()


,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


We will now only need target and stemmed content to check if a tweet is positive and negative.

In [34]:
# separating data and label
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [35]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


## Splitting


split this data into training and test data, test data for evaluation and training for training the model. x is tweet and y is label. 20% data is test and 80% data is training. Stratify y means is equal distribution of label 0 and 4.

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, stratify=y, random_state=2)

In [38]:
print(x.shape, x_train.shape, x_test.shape)

(1600000,) (1280000,) (320000,)


## Feature Extraction

Convert text into numerical

In [39]:
vectorizer = TfidfVectorizer()
#only training data gets fitted and transformed and not test data, test should only be transformed
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [40]:
print(x_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

# Training the ML (logistic regression) model

In [42]:
model = LogisticRegression(max_iter=1000)

In [43]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation

Accuracy Score

In [45]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_prediction)

In [46]:
print("Accuracy score of predicted data: ", training_data_accuracy)

Accuracy score of predicted data:  0.79871953125


Now test on test data

In [47]:
x_train_prediction = model.predict(x_test)
training_data_accuracy = accuracy_score(y_test, x_train_prediction)

In [49]:
print("Accuracy score of test data: ", training_data_accuracy)

Accuracy score of test data:  0.77668125


Model accuracy = 77.67%

# Save this trained model and use afterwards

In [50]:
import pickle

In [53]:
filename = 'trained_model.sav'
# model variable from line 42 and writing in binary form
pickle.dump(model, open(filename,'wb'))

# how to use this model for new predictions now

In [54]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))


In [55]:
x_new = x_test[200] #getting 200th data point from test data
print(y_test[200])

prediction = loaded_model.predict(x_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')


4
[4]
Positive Tweet
